In [282]:
import xlsxwriter
import pandas
import re
from collections import defaultdict

In [283]:
csv_file_name = 'results_total.csv'

def read_csv_results(csv_file_name):
    data = pandas.read_csv(csv_file_name, sep = ';', encoding='latin-1')
    data_dictionary = data.to_dict()
    return data, data_dictionary

data, data_dictionary = read_csv_results(csv_file_name)

keys = list(data_dictionary.keys())

key_status = keys[0]
key_task_type = keys[1]
key_topology_name = keys[2]
key_dataset = keys[3]
key_train_framework = keys[4]
key_inference_framework = keys[5]
key_blob_size = keys[6]
key_precision = keys[7]
key_batch_size = keys[8]
key_execution_mode = keys[9]
key_parameters = keys[10]
key_infrastructure = keys[11]
key_avgtime = keys[12]
key_latency = keys[13]
key_fps = keys[14]

print('Available keys: \n \
\t{0}, {1}, {2}, {3}, {4}, \n \
\t{5}, {6}, {7}, {8}, {9}, \n \
\t{10}, {11}, {12}, {13}, {14}'.format(key_status, key_task_type, key_topology_name, \
                                 key_dataset, key_train_framework, \
                                 key_inference_framework, key_blob_size, \
                                 key_precision, key_batch_size, \
                                 key_execution_mode, key_parameters, key_infrastructure,\
                                 key_avgtime, key_latency, key_fps))

Available keys: 
 	Status, Task type, Topology name, Dataset, Framework, 
 	Inference Framework, Input blob sizes, Precision, Batch size, Mode, 
 	Parameters, Infrastructure, Average time of single pass (s), Latency, FPS


In [284]:
excel_file_name = 'test.xlsx'
excel_sheet_name = 'Performance'

book = xlsxwriter.Workbook(excel_file_name)
sheet = book.add_worksheet(excel_sheet_name)

cell_format = book.add_format({'align': 'center', 'valign': 'vcenter', \
                               'border': 1, 'bold': True, 'text_wrap': True})
sheet.merge_range('A1:A5', key_task_type, cell_format)
sheet.merge_range('B1:B5', key_topology_name, cell_format)
sheet.merge_range('C1:C5', key_train_framework, cell_format)
sheet.merge_range('D1:D5', key_blob_size, cell_format)
sheet.merge_range('E1:E5', key_batch_size, cell_format)

In [285]:
# get unique infrastructure
infrastructure = list(set(list(data_dictionary[key_infrastructure].values())))

# get inference frameworks for each infrastructure
inference_frameworks = []
for machine in infrastructure:
    machine_inference_frameworks = []
    for key, value in data_dictionary[key_inference_framework].items():
        if data_dictionary[key_infrastructure][key] == machine and \
           (value in machine_inference_frameworks) == False:
            machine_inference_frameworks.append(value)
    inference_frameworks.append(machine_inference_frameworks)

# get devices for each inference framework
devices = []
for idx in range(len(infrastructure)):
    machine = infrastructure[idx]
    machine_inference_frameworks = inference_frameworks[idx]
    machine_framework_devices = []
    for inference_framework in machine_inference_frameworks:
        framework_devices = []
        for key, value in data_dictionary[key_parameters].items():
            pattern = re.compile(r'[.]*Device:[ ]*(?P<device_name>[\W\w]+)[,]+[.]*')
            matcher = re.match(pattern, value)
            device_name = matcher.group('device_name')
            if data_dictionary[key_infrastructure][key] == machine and \
               data_dictionary[key_inference_framework][key] == inference_framework and \
               (device_name in framework_devices) == False:
                framework_devices.append(device_name)
        machine_framework_devices.append(framework_devices)
    devices.append(machine_framework_devices)

print(infrastructure)
print(inference_frameworks)
print(devices)

['CPU: Intel(R) Core(TM) i3-8100 CPU @ 3.60GHz, CPU family: x86_64, GPU: Intel(R) Gen9 HD Graphics (iGPU), RAM size: 32757700 kB, OS family: Linux, OS version: Linux-5.4.0-73-generic-x86_64-with-Ubuntu-18.04-bionic, Python version: 3.6.9', 'CPU: Intel(R) Core(TM) i7-8700 CPU @ 3.20GHz, CPU family: x86_64, GPU: Intel(R) Gen9 HD Graphics (iGPU), RAM size: 65727428 kB, OS family: Linux, OS version: Linux-5.4.0-52-generic-x86_64-with-Ubuntu-18.04-bionic, Python version: 3.6.9', 'CPU: Intel(R) Xeon(R) Gold 6138 CPU @ 2.00GHz CPU family: x86_64, GPU: None, RAM size: 526980036 kB, OS family: Linux OS version: Linux-5.8.0-53-generic-x86_64-with-Ubuntu-18.04-bionic, Python version: 3.6.9']
[['OpenVINO DLDT'], ['OpenVINO DLDT'], ['OpenVINO DLDT']]
[[['CPU', 'GPU', 'MYRIAD']], [['CPU', 'GPU', 'MYRIAD']], [['CPU']]]


In [286]:
# get precisions for each device
precisions = []
for idx in range(len(infrastructure)):
    machine = infrastructure[idx]
    machine_inference_frameworks = inference_frameworks[idx]
    machine_precisions = []
    for idx2 in range(len(machine_inference_frameworks)):
        inference_framework = machine_inference_frameworks[idx2]
        framework_devices = devices[idx][idx2]
        framework_precisions = []
        for device in framework_devices:
            device_precisions = []
            for key, value in data_dictionary[key_parameters].items():
                pattern = re.compile(r'[.]*Device:[ ]*(?P<device_name>[\W\w]+)[,]+[.]*')
                matcher = re.match(pattern, value)
                device_name = matcher.group('device_name')
                if data_dictionary[key_infrastructure][key] == machine and \
                   data_dictionary[key_inference_framework][key] == inference_framework and \
                   device_name == device and \
                   (data_dictionary[key_precision][key] in device_precisions) == False:
                    device_precisions.append(data_dictionary[key_precision][key])
            framework_precisions.append(device_precisions)
        machine_precisions.append(framework_precisions)
    precisions.append(machine_precisions)           

print(precisions)

[[[['FP32'], ['FP32', 'FP16'], ['FP16']]], [[['FP32'], ['FP32', 'FP16'], ['FP16']]], [[['FP32']]]]


In [287]:
# get execution modes for each precision
execution_modes = []
for idx in range(len(infrastructure)):
    machine = infrastructure[idx]
    machine_inference_frameworks = inference_frameworks[idx]
    machine_modes = []
    for idx2 in range(len(machine_inference_frameworks)):
        inference_framework = machine_inference_frameworks[idx2]
        framework_devices = devices[idx][idx2]
        framework_modes = []
        for idx3 in range(len(framework_devices)):
            device = framework_devices[idx3]
            device_precisions = precisions[idx][idx2][idx3]
            framework_device_modes = []
            for precision in device_precisions:
                device_precision_modes = []
                for key, value in data_dictionary[key_parameters].items():
                    pattern = re.compile(r'[.]*Device:[ ]*(?P<device_name>[\W\w]+)[,]+[.]*')
                    matcher = re.match(pattern, value)
                    device_name = matcher.group('device_name')
                    if data_dictionary[key_infrastructure][key] == machine and \
                       data_dictionary[key_inference_framework][key] == inference_framework and \
                       device_name == device and \
                       data_dictionary[key_precision][key] == precision and \
                       (data_dictionary[key_execution_mode][key] in device_precision_modes) == False:
                        device_precision_modes.append(data_dictionary[key_execution_mode][key])
                framework_device_modes.append(device_precision_modes)
            framework_modes.append(framework_device_modes)
        machine_modes.append(framework_modes)
    execution_modes.append(machine_modes)

print(execution_modes)

[[[[['Sync', 'Async']], [['Sync', 'Async'], ['Sync', 'Async']], [['Sync', 'Async']]]], [[[['Sync', 'Async']], [['Sync', 'Async'], ['Sync', 'Async']], [['Sync', 'Async']]]], [[[['Sync', 'Async']]]]]


In [288]:
# get column indeces for each execution mode
rel_row_idx = 4
rel_col_idx = 5

col_indeces = []
for idx in range(len(infrastructure)):
    row_idx = rel_row_idx
    col_idx = rel_col_idx
    num_cols = 0
    machine = infrastructure[idx]
    machine_inference_frameworks = inference_frameworks[idx]
    col_idx3 = col_idx
    col_indeces2 = []
    for idx2 in range(len(machine_inference_frameworks)):
        machine_framework = machine_inference_frameworks[idx2]
        machine_framework_devices = devices[idx][idx2]
        col_idx2 = col_idx
        num_cols2 = 0
        col_indeces3 = []
        for idx3 in range(len(machine_framework_devices)):
            machine_framework_device = machine_framework_devices[idx3]
            framework_device_precisions = precisions[idx][idx2][idx3]
            col_idx1 = col_idx
            num_cols1 = 0
            col_indeces4 = []
            for idx4 in range(len(framework_device_precisions)):
                framework_device_precision = framework_device_precisions[idx4]
                framework_device_precision_modes = execution_modes[idx][idx2][idx3][idx4]
                col_indeces5 = []
                for idx5 in range(len(framework_device_precision_modes)):
                    framework_device_precision_mode = framework_device_precision_modes[idx5]
                    sheet.write(row_idx, col_idx + idx5, framework_device_precision_mode, cell_format)
                    col_indeces5.append(col_idx + idx5)
                sheet.merge_range(row_idx - 1, col_idx1, \
                                  row_idx - 1, col_idx1 + len(framework_device_precision_modes) - 1, \
                                  framework_device_precision, cell_format)
                col_idx += len(framework_device_precision_modes)
                col_idx1 += len(framework_device_precision_modes)
                num_cols1 += len(framework_device_precision_modes)
                num_cols2 += len(framework_device_precision_modes)
                num_cols += len(framework_device_precision_modes)
                col_indeces4.append(col_indeces5)
            sheet.merge_range(row_idx - 2, col_idx2, \
                              row_idx - 2, col_idx2 + num_cols1 - 1, \
                              machine_framework_device, cell_format)
            col_idx2 += num_cols1
            col_indeces3.append(col_indeces4)
        sheet.merge_range(row_idx - 3, col_idx3, \
                          row_idx - 3, col_idx3 + num_cols2 - 1, \
                          machine_framework, cell_format)
        col_idx3 += num_cols2
        col_indeces2.append(col_indeces3)
    sheet.merge_range(row_idx - 4, rel_col_idx, \
                      row_idx - 4, rel_col_idx + num_cols - 1, \
                      machine, cell_format)
    rel_col_idx += num_cols
    col_indeces.append(col_indeces2)

print(col_indeces)

[[[[[5, 6]], [[7, 8], [9, 10]], [[11, 12]]]], [[[[13, 14]], [[15, 16], [17, 18]], [[19, 20]]]], [[[[21, 22]]]]]


In [289]:
def remove_unused_metrics(data):
    # remove unused keys from DataFrame
    for key, value in data.copy().items():
        if key == key_avgtime or key == key_latency:
            del data[key]
    return data

def remove_failed_experiments(data):
    # transpose 2d dictionary
    experiments = data.to_dict('index')
    print('Full items: {0}'.format(len(experiments)))

    # remove failed experiments
    for key, value in experiments.copy().items():
        if value[key_status] == 'Failed':
            del experiments[key]

    print('Actual items: {0}'.format(len(experiments)))
    return experiments

def remove_unused_keys(data):
    data = remove_unused_metrics(data)  
    experiments = remove_failed_experiments(data)
    return experiments

experiments = remove_unused_keys(data)

Full items: 3528
Actual items: 3486


In [290]:
def find_idx(element, available_elements, exception_str):
    try:
        return available_elements.index(element)
    except:
        raise exception_str

def find_infrastructure_idx(infrastructure_name, available_infrastructure):
    return find_idx(infrastructure_name, available_infrastructure, \
                    'Unknown infrastructure')

def find_inference_framework_idx(framework_name, available_inference_frameworks):
    return find_idx(framework_name, available_inference_frameworks, \
                    'Unknown inference framework')

def find_device_idx(device_name, available_devices):
    return find_idx(device_name, available_devices, 'Unknown device name')

def find_precision_idx(precision, available_precisions):
    return find_idx(precision, available_precisions, 'Unknown precision')

def find_execution_mode_idx(execution_mode, available_execution_modes):
    return find_idx(execution_mode, available_execution_modes, \
                    'Unknown execution mode')

def find_column_idx(value, infrastructure, inference_frameworks, devices, \
                    precisions, execution_modes, col_indeces):
    idx1 = find_infrastructure_idx(value[key_infrastructure], infrastructure)    
    idx2 = find_inference_framework_idx(value[key_inference_framework], inference_frameworks[idx1])
    pattern = re.compile(r'[.]*Device:[ ]*(?P<device_name>[\W\w]+)[,]+[.]*')
    matcher = re.match(pattern, value[key_parameters])
    device_name = matcher.group('device_name')
    idx3 = find_device_idx(device_name, devices[idx1][idx2])
    idx4 = find_precision_idx(value[key_precision], precisions[idx1][idx2][idx3])
    idx5 = find_execution_mode_idx(value[key_execution_mode], execution_modes[idx1][idx2][idx3][idx4])
    return col_indeces[idx1][idx2][idx3][idx4][idx5]

def find_row_records(task_type, topology_name, train_framework, \
                     blob_size, batch_size, experiments, processed_records_keys):
    records_group = []
    for key, value in experiments.items():
        if key not in processed_records_keys and \
           value[key_task_type] == task_type and \
           value[key_topology_name] == topology_name and \
           value[key_train_framework] == train_framework and \
           value[key_blob_size] == blob_size and \
           value[key_batch_size] == batch_size:
            records_group.append(value)
            processed_records_keys.append(key)
    return records_group

def create_row_record(records_group, infrastructure, inference_frameworks, devices, \
                      precisions, execution_modes, col_indeces):
    fps_record = {}
    for value in records_group:
        col_idx = find_column_idx(value, infrastructure, inference_frameworks, \
            devices, precisions, execution_modes, col_indeces)
        fps_record.update( {col_idx : value[key_fps]} )
    return fps_record


processed_records_keys = []
table_records = defaultdict(list)
for key, value in experiments.items():
    if key in processed_records_keys:
        continue
    records_group = find_row_records(value[key_task_type], value[key_topology_name], \
        value[key_train_framework], value[key_blob_size], value[key_batch_size], \
        experiments, processed_records_keys)
    fps_record = create_row_record(records_group, infrastructure, inference_frameworks, \
        devices, precisions, execution_modes, col_indeces)
    record = { key_task_type : value[key_task_type], key_topology_name : value[key_topology_name], \
               key_train_framework : value[key_train_framework], key_blob_size : value[key_blob_size], \
               key_batch_size : value[key_batch_size], key_fps : fps_record }
    table_records[value[key_task_type]].append(record)

for key, value in table_records.items():
    print('{}: {}'.format(key, len(value)))

classification: 112
semantic segmentation: 6
detection: 20
object detection: 39
object recognition: 11
reidentification: 2
human pose estimation: 2
image processing: 3


In [291]:
def get_records_group(task_records, topology_name, train_framework, \
                      blob_size, processed_records_idxs):
    records_group = []
    for idx in range(len(task_records)):
        record = task_records[idx]
        if idx not in processed_records_idxs and \
           record[key_topology_name] == topology_name and \
           record[key_train_framework] == train_framework and \
           record[key_blob_size] == blob_size:
            processed_records_idxs.append(idx)
            records_group.append(record)
    return records_group

def save_table_rows(table_records, sheet):
    row_idx = 5
    for task_type, task_records in table_records.items(): # loop by tasks
        print('{}: {}'.format(task_type, len(task_records)))
        if len(task_records) <= 0:
            continue
        elif len(task_records) > 1: # print task type
            sheet.merge_range(row_idx, 0, row_idx + len(task_records) - 1, 0, task_type)
        else:
            sheet.write(row_idx, 0, task_type)
        
        processed_records_idxs = []
        for record in task_records: # loop by table records and searching for records for the same topologies
            topology_name = record[key_topology_name]
            train_framework = record[key_train_framework]
            blob_size = record[key_blob_size]
            records_group = get_records_group(task_records, topology_name, \
                                              train_framework, blob_size, \
                                              processed_records_idxs)
            topology_num_records = len(records_group)
            print('{}, {}'.format(topology_name, topology_num_records))
            if topology_num_records == 0:
                continue
            elif topology_num_records > 1:
                sheet.merge_range(row_idx, 1, row_idx + topology_num_records - 1, 1, topology_name)
                sheet.merge_range(row_idx, 2, row_idx + topology_num_records - 1, 2, train_framework)
                sheet.merge_range(row_idx, 3, row_idx + topology_num_records - 1, 3, blob_size)
            else:
                sheet.write(row_idx, 1, topology_name)
                sheet.write(row_idx, 2, train_framework)
                sheet.write(row_idx, 3, blob_size)
            for topology_record in records_group:
                sheet.write(row_idx, 4, topology_record[key_batch_size])
                for key, value in topology_record[key_fps].items():
                    sheet.write(row_idx, key, float(value))
                row_idx += 1
    
        
save_table_rows(table_records, sheet)

classification: 112
alexnet, 2
alexnet, 0
caffenet, 2
caffenet, 0
densenet-121, 2
densenet-121, 0
densenet-161, 2
densenet-161, 0
densenet-169, 2
densenet-169, 0
densenet-201, 2
densenet-201, 0
densenet-121-tf, 2
densenet-121-tf, 0
densenet-169-tf, 2
densenet-169-tf, 0
efficientnet-b0, 2
efficientnet-b0, 0
efficientnet-b0_auto_aug, 2
efficientnet-b0_auto_aug, 0
efficientnet-b0-pytorch, 2
efficientnet-b0-pytorch, 0
efficientnet-b5, 2
efficientnet-b5, 0
efficientnet-b5-pytorch, 2
efficientnet-b5-pytorch, 0
efficientnet-b7-pytorch, 2
efficientnet-b7-pytorch, 0
efficientnet-b7_auto_aug, 2
efficientnet-b7_auto_aug, 0
googlenet-v1, 2
googlenet-v1, 0
googlenet-v2, 2
googlenet-v2, 0
googlenet-v3, 2
googlenet-v3, 0
googlenet-v3-pytorch, 2
googlenet-v3-pytorch, 0
googlenet-v4-tf, 2
googlenet-v4-tf, 0
googlenet-v1-tf, 2
googlenet-v1-tf, 0
inception-resnet-v2-tf, 2
inception-resnet-v2-tf, 0
mobilenet-v1-0.25-128, 2
mobilenet-v1-0.25-128, 0
mobilenet-v1-0.50-160, 2
mobilenet-v1-0.50-160, 0
mobilene

In [292]:
book.close()